In [1]:
#import all the stuff you need
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from allensdk.brain_observatory.ecephys.ecephys_project_cache import EcephysProjectCache
from allensdk.brain_observatory.ecephys import ecephys_session

# uncomment if you wanna use the tranfer entropy measure
import sys
testdir = os.getcwd() # os.path.dirname(__file__)
import smite

# fix slow autocomplete
%config Completer.use_jedi = False
%matplotlib inline

import platform
platstring = platform.platform()

if 'Darwin' in platstring:
    # OS X 
    data_root = "/Volumes/Brain2019/"
elif 'Windows'  in platstring:
    # Windows (replace with the drive letter of USB drive)
    data_root = "E:/"
elif ('amzn1' in platstring):
    # then on AWS
    data_root = "/data/"
else:
    # then your own linux platform
    # EDIT location where you mounted hard drive
    data_root = "/media/$USERNAME/Brain2019/"
    data_root = "/run/media/tom.chartrand/Brain2019"

manifest_path = os.path.join(data_root, "dynamic-brain-workshop/visual_coding_neuropixels/2019/manifest.json")

 
cache = EcephysProjectCache.fixed(manifest=manifest_path)




ModuleNotFoundError: No module named 'aux_fun'

In [3]:

from tools import aux_fun
# from aux_fun import get_binned_spike_trains, get_binned_spike_trains_sorted
# from sanity_check import sanity_check

aux_fun.get_binned_spike_trains(333)


TypeError: get_binned_spike_trains() missing 2 required positional arguments: 'session_id' and 'stim_type'

In [ ]:
# import swdb_2019_student as swdb
# from  swdb_2019_tools import sanity_check
# from sanity_check import sanity_check

# from swdb_2019_student.sanity_check import sanity_check

In [ ]:
# session = cache.get_session_data(session_id)
# my_units = session.units
# sorted = my_units.sort_index(by='probe_vertical_position')

In [ ]:
# print(session.stimulus_conditions.stimulus_name.unique())


session_id=737581020
stim_type='drifting_gratings'#''#'spontaneous'natural_scenes''drifting_gratings
time_step = 1/100

areas1 = ['VISp']
areas2 = ['VISp','VISl','VISal','VISrl','VISam','VISpm']

T,units = get_binned_spike_trains_sorted(cache,session_id,stim_type,time_step)

In [ ]:
sanity_check(T[:,:,:])
#Ttransposed = T.transpose()
#sanity_check(Ttransposed[:,:,:10])


In [ ]:
#select a specific stim 
session = cache.get_session_data(session_id)
stim_table = session.get_presentations_for_stimulus(stim_type)
stim_ids = stim_table.index.values
if stim_type == 'natural_scenes':
    print('natural scenes')
    stim_cond = stim_conditions = stim_table.stimulus_condition_id.values
    unique_cond = stim_table.stimulus_condition_id.unique()
    frames = session.get_stimulus_parameter_values(stimulus_presentation_ids=stim_ids, drop_nulls=False)
elif stim_type == 'drifting_gratings':
    print('drifting')
    unique_cond = stim_table.orientation[stim_table.orientation != 'null'].unique()
    stim_cond = stim_conditions = stim_table.orientation.values

stim_table.orientation != 'null']



# frames = np.sort(frames)
# frames
# frames
#stim_table.orientation.unique()
#print(frames)
# unique_cond
stim_cond

In [ ]:
#Get stimulus presentation IDs for a given stimulus 

# stim_table = session.get_presentations_for_stimulus('drifting_gratings')

#And drop blank presentations

# #stim_table=stim_table[stim_table.orientation != 'null']
# stim_types = len(stim_table.orientation.unique())

# stim_orien

    
# stim_presentation_ids = stim_table[stim_table.orientation == stim_orien[0]].index.values

# #     histograms = session.presentationwise_spike_counts(
# #         bin_edges=bins,
# #         stimulus_presentation_ids=stim_presentation_ids,
# #         unit_ids=sample_units.index.values
# #     )
# print(len(stim_presentation_ids))
# print(75*8)
# T.shape


In [ ]:
# len(stim_table)
# len(stim_ids)
# unique_stims = np.unique(stim_ids)
# len(stim_conditions)
# stim_conditions
# T.shape[0]
# print(stim_conditions)
# print(stim_ids)
# all_ids = stim_conditions = stim_table.stimulus_condition_id.values
# print(all_ids)
# 5950/len(unique_cond)
# T.shape

In [ ]:
stim_cond

In [ ]:
# for each stimulus in frames average across trials 
num_stims = len(unique_cond)
Avg_psth = np.zeros((T.shape[0],T.shape[1],num_stims))
for ix,cond in enumerate(unique_cond):
    #print(ix)
    idx_in = stim_cond == cond
    #print(sum(idx_in))
    Avg_psth[:,:,ix] = np.nanmean(T[:,:,idx_in],2)


In [ ]:
# np.nanmean(T[:,:,idx_in],2)
#Ttranp = np.transpose(Avg_psth)
#sanity_check(Avg_psth[:,:,:10])
#sanity_check(Ttranp[:,:,:16],ncols=8)

#print(Avg_psth.shape)
#print(TT.shape)


# units.head()
# units.structure_acronym.unique()
Avg_psth.shape


In [ ]:
units['Row_number'] = np.arange(len(units))
unitIDs1 = []
unitIDs2 = []
for index in units.index:
    structure_acronym = units.structure_acronym[index]
#     print(index, structure_acronym)
    if structure_acronym in areas1:
        unitIDs1.append(index)    
    if structure_acronym in areas2:
        unitIDs2.append(index)    

unit_ind1 = units.Row_number[unitIDs1].values
unit_ind2 = units.Row_number[unitIDs2].values

N1 = len(unitIDs1)
N2 = len(unitIDs2)

In [ ]:
# print(T.shape)
# T[0,1:10,1]
# T.shape[0]
#unit_ind1
print(N1)
print(N2)


In [ ]:
XX = Avg_psth
import scipy.stats as stats
import statsmodels as statsmodels
from statsmodels import tsa
from statsmodels.tsa import stattools
corr_mat = np.zeros([N1,N2,np.minimum(10,XX.shape[2])])
for itrial in range(np.minimum(10,XX.shape[2])):
    print(itrial)
#itrial=2
    for i in range(N1):
        for j in range(N2):
#             corr_mat[i,j,itrial] = stats.pearsonr(XX[unit_ind1[i],:,itrial],XX[unit_ind2[j],:,itrial])[0]
            #print(j)

            # uncomment this for granger casuality
            #zipped_array  = np.array(list(zip(TT[i,:,itrial],TT[j,:,itrial])))
            #granger_test_result = statsmodels.tsa.stattools.grangercausalitytests(zipped_array,maxlag=10);
            #corr_mat[i,j,itrial]=granger_test_result[10][0]['ssr_ftest'][0]
            
            symX = smite.symbolize(XX[unit_ind1[i],:,itrial],3)
            symY = smite.symbolize(XX[unit_ind2[j],:,itrial],3)

            TXY = smite.symbolic_transfer_entropy(symX, symY)
            TYX = smite.symbolic_transfer_entropy(symY, symX)
            corr_mat[i,j,itrial]=TYX - TXY
            #corr_mat[j,i,itrial]=TXY - TYX   
            
import pickle

def save_object(obj, filename):
     with open(filename, 'wb') as output:  # Overwrites any existing file.
        pickle.dump(obj, output, pickle.HIGHEST_PROTOCOL)

save_object(corr_mat, str('corr_mat_' + stim_type + 'session_' + str(session_id) +'.pkl'))   



In [ ]:
sanity_check(corr_mat,'units 1','units 2','stimulus',ncols=3)


In [ ]:
# image_num = int(stim_table.frame[stim_table.stimulus_condition_id == unique_cond[4]].unique())
# image_template = cache.get_natural_scene_template(image_num)

# plt.imshow(image_template, cmap='gray')

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=20)
pca.fit(corr_mat[:,:,0])

In [ ]:
pca.explained_variance_
print(pca.explained_variance_ratio_)
pca.components_ #eigenvectors
X_project = pca.fit_transform(corr_mat[:,:,0])
max_pc = 3

In [ ]:
X_project.shape

In [ ]:
plt.imshow(X_project)

In [ ]:
plt.plot(pca.components_[0,:])

In [ ]:
sort_id = np.argsort(X_project[:,0])
sanity_check(corr_mat[sort_id,:,0])

In [ ]:
sanity_check(corr_mat[:,:,0])

In [ ]:
from sklearn.cluster import KMeans
from scipy import stats

kmeans = KMeans(n_clusters=3).fit(X_project[:,:3])
unsupervised_y_hat = kmeans.labels_

# class_0, __ = stats.mode(unsupervised_y_hat[y==0])
# class_1, __ = stats.mode(unsupervised_y_hat[y==1])
# class_2, __ = stats.mode(unsupervised_y_hat[y==2])

# sorted_unsup_y_hat = unsupervised_y_hat.copy()
# sorted_unsup_y_hat[unsupervised_y_hat==class_0] = 0
# sorted_unsup_y_hat[unsupervised_y_hat==class_1] = 1
# sorted_unsup_y_hat[unsupervised_y_hat==class_2] = 2

# plot_test_performance(X_project,y,sorted_unsup_y_hat)
plt.scatter(X_project[unsupervised_y_hat==0,0],X_project[unsupervised_y_hat==0,1])
plt.scatter(X_project[unsupervised_y_hat==1,0],X_project[unsupervised_y_hat==1,1])
plt.scatter(X_project[unsupervised_y_hat==2,0],X_project[unsupervised_y_hat==2,1])


In [ ]:
unsupervised_y_hat


In [ ]:
sort_id = np.argsort(unsupervised_y_hat)
sanity_check(corr_mat[sort_id,:,0])

# fig, ax = plt.subplots()
# lfp_plot = lfp.loc[dict(time=slice(5,20))]
# x, y = lfp_plot.time, range(len(lfp_plot.channel))
# plt.pcolormesh(x, y, lfp_plot.values.T)
# plt.colorbar(ax=ax)

# ax.set_xlabel("time (s)")

# # include the structure data
# structure_acronyms, intervals = session.channel_structure_intervals(lfp.channel.values)
# ax.set_yticks(intervals)
# interval_midpoints = [ (aa + bb) / 2 for aa, bb in zip(intervals[:-1], intervals[1:])]
# ax.set_yticks(interval_midpoints, minor=True)
# ax.set_yticklabels(structure_acronyms, minor=True)
# plt.tick_params("y", which="major", labelleft=False, length=40)
# plt.show()

In [ ]:

units_V1 = units.loc[unitIDs1]
units_V1['cluster']=unsupervised_y_hat

save_object(units_V1, str('V1_clusters' + stim_type + 'session_' + str(session_id) +'.pkl'))   



In [ ]:
# def scary_function(a, b):
#     return a + b

# print(scary_function(0, 1))
# print(scary_function("a", "b"))
# print(scary_function(np.array([1, 2, 3]), np.array([4, 5, 6])) )

# aa= 'a', 4
# print(aa, type(aa))

# bb= [np.array([1, 2, 3]), np.array([4, 5, 6]), "hi"]
# print(bb, type(bb))

In [ ]:
# #zipped_array = np.array(list(zip(TT[0,:,0],TT[1,:,0])))
# x1 = np.random.rand(100)
# y1 = np.concatenate([x1[3:], x1[:3]])
# print(x1.shape)

# print(y1.shape)
# zipped_array  = np.array(list(zip(x1,y1)))
# zipped_array.shape

In [ ]:
# """
# Statistical tools for time series analysis
# """
# import statsmodels as statsmodels
# from statsmodels import tsa
# from statsmodels.tsa import stattools

# granger_test_result = statsmodels.tsa.stattools.grangercausalitytests(zipped_array,maxlag=15);
# print(granger_test_result[key][0])
# #granger_test_result = grangercausalitytests(data[:, 1::-1], maxlag=12, verbose#=False)
                                            
# # optimal_lag = -1
# # F_test = -1.0
# # for key in granger_test_result.keys():
# #      _F_test_ = granger_test_result[key][0]['params_ftest'][0]
# # #     if _F_test_ > F_test:
# #         F_test = _F_test_
# #         optimal_lag = key


In [ ]:
# print(granger_test_result[1][0]['ssr_ftest'])

# print(type(granger_test_result))